# Understand Number Space - Similar Mode

Our `NumberSpace` has a SIMILAR mode too, enabling search for similar numbers. 

Check MinMax mode [here](number_embedding_minmax.ipynb).

In [1]:
%pip install superlinked==23.0.1

In [2]:
from superlinked import framework as sl

## Similar mode

In this case we prefer numbers that are similar to the number we are going to query with.

In [3]:
class Person(sl.Schema):
    id: sl.IdField
    height: sl.Integer  # set height in cms for example


person = Person()

height_space = sl.NumberSpace(
    number=person.height, min_value=100, max_value=220, mode=sl.Mode.SIMILAR
)  # height out of this range is considered an outlier in our use case
person_index = sl.Index(height_space)

In [4]:
source: sl.InMemorySource = sl.InMemorySource(person)
executor = sl.InMemoryExecutor(sources=[source], indices=[person_index])
app = executor.run()

In [5]:
source.put([{"id": "person1", "height": 175}, {"id": "person2", "height": 180}])

In [6]:
query = sl.Query(person_index).find(person).similar(height_space, 178).select_all()

In [7]:
result = app.query(query)
sl.PandasConverter.to_pandas(result)

,height,id,similarity_score
0,180,person2,0.999657
1,175,person1,0.999229


As expected, the person 180 cm high comes first (as their height is closer to the queried number, 178), and the person with height of 175 cm comes second - while the difference in score is extremely low.